In [0]:
%fs ls /mnt/lending_club/landing/loan_defaulters_df/csv/single

path,name,size,modificationTime
dbfs:/mnt/lending_club/landing/loan_defaulters_df/csv/single/_SUCCESS,_SUCCESS,0,1702883183000
dbfs:/mnt/lending_club/landing/loan_defaulters_df/csv/single/_committed_2898080382008637628,_committed_2898080382008637628,113,1702883183000
dbfs:/mnt/lending_club/landing/loan_defaulters_df/csv/single/_started_2898080382008637628,_started_2898080382008637628,0,1702883178000
dbfs:/mnt/lending_club/landing/loan_defaulters_df/csv/single/part-00000-tid-2898080382008637628-d409c413-9f62-4986-889e-aa2bbe9225c0-232-1-c000.csv,part-00000-tid-2898080382008637628-d409c413-9f62-4986-889e-aa2bbe9225c0-232-1-c000.csv,211765942,1702883183000


In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, StringType, FloatType, LongType, TimestampType
from pyspark.sql.functions import regexp_replace, concat_ws
from pyspark.sql.functions import when, col, length
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import avg

In [0]:
loans_defaulters_schema = StructType([
    
    StructField("member_id", StringType(), True)
    ,StructField("deling_2yrs", FloatType(), True)
    ,StructField("delinq_amount", FloatType(), True)
    ,StructField("pub_rec", FloatType(), True)
    ,StructField("pub_rec_bankruptcies", FloatType(), True)
    ,StructField("inq_last_6mths", FloatType(), True)
    ,StructField("total_rec_late_fee", FloatType(), True)
    ,StructField("mths_since_last_delinq", FloatType(), True)
    ,StructField("mths_since_last_record", FloatType(), True)
])

In [0]:
loans_defaulters_df = spark.read\
    .format("csv")\
    .option("header", True)\
    .schema(loans_defaulters_schema)\
    .load("dbfs:/mnt/lending_club/landing/loan_defaulters_df/csv/single/part-00000-tid-2898080382008637628-d409c413-9f62-4986-889e-aa2bbe9225c0-232-1-c000.csv")


loans_defaulters_df.show()

+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|deling_2yrs|delinq_amount|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|af2c01919a67ad070...|        0.0|          0.0|    0.0|                 0.0|           1.0|               0.0|                  30.0|                  NULL|
|51cf0089ac3e1beeb...|        1.0|          0.0|    0.0|                 0.0|           4.0|               0.0|                   6.0|                  NULL|
|a03d0c087a03631e3...|        0.0|          0.0|    0.0|                 0.0|           0.0|               0.0|                  NULL|                  NULL|
|edb5d90fadb8d0754...|        0.0|          0.0|    

In [0]:
loans_defaulters_delind2yrs_df = loans_defaulters_df.withColumn(
    "deling_2yrs",
    col("deling_2yrs").cast("integer")
).fillna(0, subset = ["deling_2yrs"])

In [0]:
loans_defaulters_delind2yrs_df.show()

+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|deling_2yrs|delinq_amount|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|af2c01919a67ad070...|          0|          0.0|    0.0|                 0.0|           1.0|               0.0|                  30.0|                  NULL|
|51cf0089ac3e1beeb...|          1|          0.0|    0.0|                 0.0|           4.0|               0.0|                   6.0|                  NULL|
|a03d0c087a03631e3...|          0|          0.0|    0.0|                 0.0|           0.0|               0.0|                  NULL|                  NULL|
|edb5d90fadb8d0754...|          0|          0.0|    

In [0]:
loans_defaulters_delind2yrs_ingested_df = loans_defaulters_delind2yrs_df.withColumn(
    "ingested_date", current_timestamp()
)
loans_defaulters_delind2yrs_ingested_df.show()

+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+--------------------+
|           member_id|deling_2yrs|delinq_amount|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|       ingested_date|
+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+--------------------+
|af2c01919a67ad070...|          0|          0.0|    0.0|                 0.0|           1.0|               0.0|                  30.0|                  NULL|2023-12-20 09:48:...|
|51cf0089ac3e1beeb...|          1|          0.0|    0.0|                 0.0|           4.0|               0.0|                   6.0|                  NULL|2023-12-20 09:48:...|
|a03d0c087a03631e3...|          0|          0.0|    0.0|                 0.0|           0.0|             

In [0]:
loans_defaulters_delind2yrs_ingested_df.show(2)

+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+--------------------+
|           member_id|deling_2yrs|delinq_amount|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|       ingested_date|
+--------------------+-----------+-------------+-------+--------------------+--------------+------------------+----------------------+----------------------+--------------------+
|af2c01919a67ad070...|          0|          0.0|    0.0|                 0.0|           1.0|               0.0|                  30.0|                  NULL|2023-12-20 09:56:...|
|51cf0089ac3e1beeb...|          1|          0.0|    0.0|                 0.0|           4.0|               0.0|                   6.0|                  NULL|2023-12-20 09:56:...|
+--------------------+-----------+-------------+-------+--------------------+--------------+-------------

In [0]:
defaulters_df = loans_defaulters_delind2yrs_ingested_df.select("member_id").filter("pub_rec >0.0").filter("pub_rec_bankruptcies>0.0").filter("inq_last_6mths>0.0")

In [0]:
defaulters_df.write \
.format("csv") \
.mode("overwrite") \
.option("path", "/mnt/lending_club/staging/loans_defaulters/deafulters/non_partitioned/csv") \
.save()

In [0]:
defaulters_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/mnt/lending_club/staging/loans_defaulters/deafulters/non_partitioned/parquet") \
.save()

In [0]:
loans_defaulters_delind2yrs_ingested_df = loans_defaulters_delind2yrs_ingested_df.filter("deling_2yrs > 0").filter("mths_since_last_delinq > 0.0")

In [0]:
loans_defaulters_delind2yrs_ingested_df.write \
.format("csv") \
.mode("overwrite") \
.option("path", "/mnt/lending_club/staging/loans_defaulters/non_deafulters/non_partitioned/csv") \
.save()

In [0]:
loans_defaulters_delind2yrs_ingested_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/mnt/lending_club/staging/loans_defaulters/non_deafulters/non_partitioned/parquet") \
.save()